In [1]:
import pandas as pd
import numpy as np
import os
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import col

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow
import click

import spacy
import nltk

from nltk.tokenize import word_tokenize
from scispacy.abbreviation import AbbreviationDetector

import string

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

nltk.download('all')
%matplotlib inline 

[nltk_data] Downloading collection 'all'
[nltk_data] | 
[nltk_data] | Downloading package abc to /root/nltk_data...
[nltk_data] | Unzipping corpora/abc.zip.
[nltk_data] | Downloading package alpino to /root/nltk_data...
[nltk_data] | Unzipping corpora/alpino.zip.
[nltk_data] | Downloading package averaged_perceptron_tagger to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] | Downloading package averaged_perceptron_tagger_ru to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping
[nltk_data] | taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data] | Downloading package basque_grammars to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping grammars/basque_grammars.zip.
[nltk_data] | Downloading package biocreative_ppi to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/biocreative_ppi.zip.
[nltk_data] | Downloading package bllip_wsj_no_aux to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping models/bllip_wsj_no_aux.zip.
[nltk_data] | Downloading package book_grammars to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping grammars/book_grammars.zip.
[nltk_data] | Downloading package brown to /root/nltk_data...
[nltk_data] | Unzipping corpora/brown.zip.
[nltk_data] | Downloading package brown_tei to /root/nltk_data...
[nltk_data] | Unzipping corpora/brown_tei.zip.
[nltk_data] | Downloading package cess_cat to /root/nltk_data...
[nltk_data] | Unzipping corpora/cess_cat.zip.
[nltk_data] | Downloading package cess_esp to /root/nltk_data...
[nltk_data] | Unzipping corpora/cess_esp.zip.
[nltk_data] | Downloading package chat80 to /root/nltk_data...
[nltk_data] | Unzipping corpora/chat80.zip.
[nltk_data] | Downloading package city_database to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/city_database.zip.
[nltk_data] | Downloading package cmudict to /root/nltk_data...
[nltk_data] | Unzipping corpora/cmudict.zip.
[nltk_data] | Downloading package comparative_sentences to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/comparative_sentences.zip.
[nltk_data] | Downloading package comtrans to /root/nltk_data...
[nltk_data] | Downloading package conll2000 to /root/nltk_data...
[nltk_data] | Unzipping corpora/conll2000.zip.
[nltk_data] | Downloading package conll2002 to /root/nltk_data...
[nltk_data] | Unzipping corpora/conll2002.zip.
[nltk_data] | Downloading package conll2007 to /root/nltk_data...
[nltk_data] | Downloading package crubadan to /root/nltk_data...
[nltk_data] | Unzipping corpora/crubadan.zip.
[nltk_data] | Downloading package dependency_treebank to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/dependency_treebank.zip.
[nltk_data] | Downloading package dolch to /root/nltk_data...
[nltk_data] | Unzipping corpora/dolch.zip.
[nltk_data] | Downloading package europarl_raw to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/europarl_raw.zip.
[nltk_data] | Downloading package extended_omw to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/extended_omw.zip.
[nltk_data] | Downloading package floresta to /root/nltk_data...
[nltk_data] | Unzipping corpora/floresta.zip.
[nltk_data] | Downloading package framenet_v15 to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/framenet_v15.zip.
[nltk_data] | Downloading package framenet_v17 to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/framenet_v17.zip.
[nltk_data] | Downloading package gazetteers to /root/nltk_data...
[nltk_data] | Unzipping corpora/gazetteers.zip.
[nltk_data] | Downloading package genesis to /root/nltk_data...
[nltk_data] | Unzipping corpora/genesis.zip.
[nltk_data] | Downloading package gutenberg to /root/nltk_data...
[nltk_data] | Unzipping corpora/gutenberg.zip.
[nltk_data] | Downloading package ieer to /root/nltk_data...
[nltk_data] | Unzipping corpora/ieer.zip.
[nltk_data] | Downloading package inaugural to /root/nltk_data...
[nltk_data] | Unzipping corp

In [2]:
# Table 1
spark.sql("use standaardtabellen")
NASA_totaal = spark.sql("select * from nasastandaardtabel")
NASA_totaal = NASA_totaal.select(col("NASA_NR"), col("PRIMARY_EAN").alias("ARTIKEL_EAN"), col("GLN").alias("gln_nasa"), col("SCHAPSTICKER_OMSCHRIJVING"),col("ASSGROEP_OMSCHRIJVING"), col("KASSABON_OMSCHRIJVING"), col("LANGE_OMSCHRIJVING"), col("EENHEID")
                                , col("INHOUD"))
# Table 2
GLNAH = spark.read.format('delta').load('/mnt/Prd_adls/Conformed/TIMS/TradeItem/ahTradeItem/Data/').filter('__DeletedFlag == 0')
GLNAH = GLNAH.select(col("gtin").alias("gtin1"), col("gln").alias('gln1'), col("glnAH"), col("ndDepartment"))

# Table 3
spark.read.format('delta').load('/mnt/Prd_adls/Conformed/TIMS/TradeItem/tradeItemInfo/Data/').filter('__DeletedFlag == 0').registerTempTable('tradeItemInfo')
tradeItemInfo = spark.sql("select gtin, gln, coalesce (tradeItemInfo.mvlang_tradeItemDescMod_tradeItemDescInfo_tradeItemDesc['nl'], tradeItemInfo.mvlang_tradeItemDescMod_tradeItemDescInfo_tradeItemDesc['en']) as TradeItemDescr, coalesce(lang_tradeItemDescMod_tradeItemDescInfo_functionalName['nl'], lang_tradeItemDescMod_tradeItemDescInfo_functionalName['en']) as functionalName, coalesce(mvlang_tradeItemDescMod_tradeItemDescInfo_labelDesc['nl'], mvlang_tradeItemDescMod_tradeItemDescInfo_labelDesc['en']) as labelDescription, coalesce(mvlang_tradeItemDescMod_tradeItemDescInfo_regulatedProdName['nl'], mvlang_tradeItemDescMod_tradeItemDescInfo_regulatedProdName['en']) as regulatedProdName, coalesce(lang_foodAndBeverageIngredientMod_ingredientStatement['nl'], lang_foodAndBeverageIngredientMod_ingredientStatement['en']) as ingredientStatement from tradeItemInfo")
# coalesce, returned de eerste non-null value in een list


# Join several Tables
NASA_totaal = NASA_totaal.join(GLNAH, (NASA_totaal.ARTIKEL_EAN == GLNAH.gtin1) & (NASA_totaal.gln_nasa == GLNAH.gln1), how = "left")
NASA_totaal = NASA_totaal.join(tradeItemInfo, (NASA_totaal.ARTIKEL_EAN == tradeItemInfo.gtin) & (NASA_totaal.gln_nasa == tradeItemInfo.gln), how = "left")
print((NASA_totaal.count(),len(NASA_totaal.columns))) 
NASA_totaal = NASA_totaal.filter(NASA_totaal.glnAH == True)

# Select Features
NASA = NASA_totaal.select("ARTIKEL_EAN", "gln_nasa","glnAH","regulatedProdName","functionalName", "labelDescription","TradeItemDescr","ingredientStatement",)
NASA = NASA.withColumnRenamed("ARTIKEL_EAN", "gtin") # hernoem naar gtin, zodat we department kunnen mergen.
NASA = NASA.withColumnRenamed("gln_nasa", "gln") # hernoem naar gtin, zodat we department kunnen mergen.
print("Shape van DF (row, columns) ",(NASA.count(),len(NASA.columns))) # Shape of dataframe

# Arrays naar strings
NASA = NASA.withColumn("regulatedProdName", concat_ws(" ", "regulatedProdName"))
NASA = NASA.withColumn("functionalName", concat_ws(" ", "functionalName"))
NASA = NASA.withColumn("labelDescription", concat_ws(" ", "labelDescription"))
NASA = NASA.withColumn("TradeItemDescr", concat_ws(" ", "TradeItemDescr"))
NASA = NASA.withColumn("ingredientStatement", concat_ws(" ", "ingredientStatement"))

display(NASA[1])

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
 warnings.warn(
(48744, 20)
Shape van DF (row, columns) (35075, 8)

gtin,gln,glnAH,regulatedProdName,functionalName,labelDescription,TradeItemDescr,ingredientStatement
3159470001424,38000000072,true,"Ontbijtgranen van tarwezemelen, van nature uit rijk aan vezels (27%), verrijkt met vitaminen (B₁, B₂, niacine, B₆, foliumzuur, B₁₂) en ijzer.",Ontbijtgranen,Kellogg's All-Bran Plus 500 g,Kellogg's ALL BRAN FLAKES 500g,"Ingrediënten: TARWEZEMELEN (87%), suiker, GERSTEMOUTAROMA, zout, Vitaminen en mineralen: vitaminen (niacine, B6, B2, B1, foliumzuur, B12), ijzer. Allergenen: zie ingrediëntenlijst, in vet gemarkeerd."
3159470005347,38000000072,true,"Assortiment van individuele verpakkingen van ontbijtgranen op basis van maïs, rijst en tarwe.",Ontbijtgranen,Kellogg's Variety Mini Packs 8 Stuks 215 g,Kellogg's Variety 220g,"Ingrediënten: Granen (maïs, rijst, TARWEMEEL), suiker, glucosesiroop, cacaopoeder, cacaopoeder met verlaagd vetgehalte, zout, GERSTEMOUTEXTRACT, honing, cacaomassa, maïsvezels, aroma's, natuurlijk aroma, kaneel, kleurstof (carotenen), magere MELKPOEDER, SOJA-eiwit isolaat, vitamine D, vitamine B1, vitamine B2, niacine, vitamine B6, foliumzuur, vitamine B12, ijzer Bevat mogelijk: pindanoten, gluten van andere granen. Voor allergenen: zie ingrediënten in vet gemarkeerd."
3159470200087,38000000072,true,"RIJST-, VOLKORENTARWE- EN GERSTVLOKKEN MET GEVRIESDROOGDE RODE VRUCHTEN, VERRIJKT MET VITAMINEN EN MINERALEN.",Ontbijtgranen,Kellogg's Special K Red Fruits 300g,Kellogg's SPECIAL K Red Fruit 300g,"Ingrediënten: Rijst (44%), VOLKORENTARWE (35%), suiker, GERST (4.5%), gevriesdroogd fruit (4.5%) (aardbei, kers), gemout GERSTEMEEL (3.5%), GERSTEMOUTAROMA, zout, Vitaminen en mineralen: vitaminen (niacine, B2, B1, B6, foliumzuur, D, B12), ijzer, zink. Bevat mogelijk melk. Voor allergenen: zie ingrediënten in vet gemarkeerd."
3159470202166,38000000072,true,"RIJST-, VOLKORENTARWE- EN GERSTVLOKKEN MET PURE CHOCOLADEKRULLEN, VERRIJKT MET VITAMINEN EN MINERALEN.",Ontbijtgranen,Kellogg's Special K Pure Chocolade 300 g,Kellogg's SPECIAL K Dark Chocolate 300g,"Ingrediënten: Rijst (39%), VOLKORENTARWE (31%), chocolade (14%) (suiker, cacaomassa, MELKVET, cacaoboter, emulgator {SOJALECITHINE}), suiker, GERST (4%), gemout GERSTEMEEL (3%), GERSTEMOUTAROMA, zout, Vitaminen en mineralen: Vitaminen (niacine, B2, B1, B6, foliumzuur, D, B12) ijzer, zink Voor allergenen: zie ingrediënten in vet gemarkeerd."
4003994152508,38000000072,true,"VOLKORENGRANENRINGETJES MET HONING, VERRIJKT MET VITAMINEN EN IJZER.",Ontbijtgranen,Kellogg's Miel Pops Loops 375 g,Kellogg's HONEY LOOPS 375g,"Ingrediënten: Granenmeel (74%) (VOLKORENTARWE, VOLKORENHAVER, VOLKORENGERST, VOLKORENROGGE), suiker, honing (3%), glucosesiroop, zout, tricalciumfosfaat, kleurstof (carotenen), antioxidanten (ascorbylpalmitaat, alfatocoferol), Vitaminen en mineralen: vitaminen (niacine, B6, B2, B1, foliumzuur, B12, D), ijzer. Voor allergenen: zie ingrediënten in vet gemarkeerd."
4003994637159,38000000072,true,"GEPOFTE EN GEKARAMELISEERDE TARWEKORRELS, VERRIJKT MET VITAMINEN EN IJZER.",Ontbijtgranen,Kellogg's Smacks 375 g,Kellogg's SMACKS 375g,"Ingrediënten: TARWE (55%), suiker, glucosesiroop, Vitaminen en mineralen: vitaminen (niacine, B6, B2, B1, foliumzuur, D, B12), ijzer. Voor allergenen: zie ingrediënten in vet gemarkeerd."
4003994655290,38000000072,true,"KROKANTE ONTBIJTGRANEN VAN TARWE MET CHOCOLADE, VERRIJKT MET VITAMINEN EN IJZER.",Ontbijtgranen,Kellogg's Coco Pops Chocos 375 g,Kellogg's CHOCOS 375g,"Ingrediënten: TARWEMEEL (66%), suiker, chocoladepoeder (11%) (suiker, cacaopoeder), cacaopoeder met verlaagd vetgehalte, glucosesiroop, zout, kaneel, natuurlijk aroma, Vitaminen en mineralen: vitaminen (niacine, B6, B2, B1, foliumzuur, B12, D), ijzer. Voor allergenen: zie ingrediënten in vet gemarkeerd."
5000127061828,38000000072,true,"VOLKORENTARWEVLOKKEN MET ZEMELEN, VERRIJKT MET VITAMINEN EN IJZER.",Ontbijtgranen,Kellogg's All-Bran Flakes 375 g,Kellogg's ALL BRAN FLAKES 375g,"Ingrediënten: VOLKORENTARWE (59%), TARWEZEMELEN (18%), VOLKORENBLOEM,

In [3]:
NASA = NASA.filter(NASA.glnAH == True)
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")
df_pandas = NASA.toPandas()

df_pandas = df_pandas.astype({"regulatedProdName":"string", "functionalName":"string","labelDescription":"string","TradeItemDescr":"string","ingredientStatement":"string"}) # object to string
df_NLP = df_pandas[['gtin','gln','regulatedProdName','functionalName','labelDescription','TradeItemDescr','ingredientStatement']].copy()

In [4]:
text_features = ['regulatedProdName','functionalName','labelDescription','TradeItemDescr','ingredientStatement']
for TEXT_F in text_features:
    df_NLP[str(TEXT_F)] = df_NLP[TEXT_F].apply(lambda x: len(x.split()))

In [5]:
display(df_NLP.head())

gtin,gln,regulatedProdName,functionalName,labelDescription,TradeItemDescr,ingredientStatement
3159470001424,38000000072,21,1,5,5,23
3159470005347,38000000072,13,1,8,3,54
3159470200087,38000000072,13,1,6,6,41
3159470202166,38000000072,12,1,7,6,41
4003994152508,38000000072,8,1,6,4,37
4003994637159,38000000072,9,1,4,3,24
4003994655290,38000000072,11,1,6,3,36
5000127061828,38000000072,8,1,5,5,31
5000127077225,38000000072,11,1,6,7,42
5050083251146,38000000072,11,1,7,6,47


Out[14]:

Text(0.5, 0, 'Number of tokens')

In [6]:
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,7))
plt.rcParams.update({'font.size': 20})
medianprops = dict(linestyle='-', linewidth=2, color='orange')
bbplot = df_NLP.boxplot(column=['regulatedProdName','functionalName','labelDescription','TradeItemDescr','ingredientStatement'], showfliers=False, vert=False, grid=False, patch_artist=True, medianprops=medianprops)


bbplot.set_ylabel('Text Features')
bbplot.set_xlabel('Number of tokens')